### 1. 简介
手写数字识别是深度学习的经典入门项目，非常适合初学者系统掌握深度学习的基本流程。

本文将基于 PyTorch 框架，采用 CNN（卷积神经网络）实现手写数字识别任务。

### 2. 图像在计算机中的存储
数字图像在计算机中以矩阵形式存储。以灰度图（灰色的图）为例，它是一个二维矩阵，每个元素对应一个像素点，
其整数范围是0到255：0代表纯黑，255代表纯白，中间值代表不同深度的灰色。

而对于彩色图像，则采用三通道RGB模型。图像由Red（红）、Green（绿）、Blue（蓝）三个二维矩阵叠加而成，
每个通道矩阵同样使用0-255的整数值来表示该色光的强度。通过三个通道在空间上不同亮度的组合，即可混合出
丰富的色彩。因此，一张彩色图像在计算机中可以表示为一个三维张量，其中一维代表通道，另两位代表图像的高
度和宽度。


### 3. 数据集
本文采用Kaggle比赛Digit Recognizer提供的MNIST数据集。该数据集包含数万张训练图像及28000张测试图像。
每张图像为28x28像素的灰度图。在数据集的CSV文件中，每一行代表一个样本，首列为该图像所对应的数字真值标签，
随后的784列（28x28）则按行展开为该图像的像素灰度值。格式如下所示：

label,pixel0,pixel1,...,pixel783

1,0,0,...,100

该数据由28×28像素矩阵展平为一维数组得到。

### 4. 数据预处理
pandas是Python的核心数据处理库，可用于数据读取与分析。

In [ ]:
import pandas as pd

# 加载数据
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

# 显示前五行
train.head()

In [ ]:
# 取标签列作为输出（Y）
Y_train = train['label']

# 丢弃标签列，将剩余数据作为输入（X）
X_train = train.drop(labels=['label'], axis = 1)

# 删除train，释放内存
del train

# 查看每个值的数量
Y_train.value_counts()

In [ ]:
# 检查是否存在缺失值
X_train.isnull().any().describe()

In [ ]:
test.isnull().any().describe()

In [ ]:
# 进行灰度归一化，将像素值从[0, 255]的区间缩放至[0, 1]的范围。
X_train = X_train / 255.0
test = test / 255.0

In [ ]:
# 查看训练集的形状
X_train.shape

X_train训练集形状为(42000, 784)，意味着具有42000个样本，每个样本具有784个像素值

In [ ]:
# 将图像尺寸重塑为28×28，以便输入后续的卷积层进行特征提取。
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

使用 reshape(-1,28,28,1) 方法将数据重塑后，原始的784维特征向量被转换为28×28×1的三维张量。
其中，参数-1的作用是自动计算样本数量，从而保持数据总量的不变。

In [ ]:
# 再次查看训练集形状
X_train.shape

sklearn是一个广泛应用于机器学习的Python库。
我们使用其中的train_test_split方法，将原始训练集随机划分为训练集和验证集，
以进行模型训练与评估。

In [ ]:
from sklearn.model_selection import train_test_split

# 设置随机数种子，便于复现。
random_seed = 2

# 九成用于训练，一成用于验证。
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.1, random_state=random_seed)

In [ ]:
# 查看训练集类型
type(X_train)

In [ ]:
X_train.shape

In [ ]:
type(Y_train)

In [ ]:
Y_train = Y_train.values

X_train的类型变成了numpy.ndarray，这是NumPy库提供的核心多维数组数据结构。NumPy是Python中用于科学计算的基础库。


Matplotlib是Python中常用的绘图库。我们使用它来可视化训练集中的第一张样本图像。

In [ ]:
import matplotlib.pyplot as plt

# 设置Matplotlib在Notebook中内嵌显示图形，无需单独调用plt.show()
%matplotlib inline

# 显示第一个训练样本的第一个通道
plt.imshow(X_train[0][:,:,0])

### 5. 模型设计与训练

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2)
        self.dropout1 = nn.Dropout(0.25)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.dropout2 = nn.Dropout(0.25)
        
        self.fc1 = nn.Linear(64 * 7 * 7, 256)
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2) # 默认stride = kernel_size
        x = self.dropout1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size=2)
        x = self.dropout2(x)
        
        x = x.reshape(x.size(0), -1) # 展平
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)

        return x
    
model = CNN()

In [ ]:
model

为提升训练效果，需配置优化算法与动态学习率调整机制，这是现代深度学习模型训练中必不可少的一环。

In [ ]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

配置训练轮次与批处理规模。

In [ ]:
epochs = 1
batch_size = 86

基于PyTorch框架，可利用torchvision.transforms组合多种数据增强方法，
配合Dataset类系统化提升模型泛化能力。

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(10), # 旋转
    transforms.RandomResizedCrop( # 裁剪
        size=28,
        scale=(0.9, 1.1),
        ratio=(0.9, 1.1)
    ),
    transforms.RandomAffine(
        degrees=0,
        translate=(0.1, 0.1), # 水平和垂直平移
    ),
    transforms.ToTensor()
])

# 自定义数据集，用于数据增强
class AugmentedDataset(Dataset):
    def __init__(self, data, targets, transform):
        self.data = data
        self.targets = targets
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.transform(self.data[index])
        y = self.targets[index]
        
        return x, y

In [ ]:
x_train_tensor = torch.FloatTensor(X_train).permute(0, 3, 1, 2)
y_train_tensor = torch.LongTensor(Y_train)

train_dataset = AugmentedDataset(x_train_tensor, y_train_tensor, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
len(train_loader)

针对十分类任务的输出特性，选用交叉熵作为损失函数。

In [ ]:
criterion = nn.CrossEntropyLoss()

开始训练。
我们使用nvidia gpu配合cuda进行训练加速。

In [ ]:
model.to(device='cuda')

model.train()

for epoch in range(epochs):
    epoch_loss = 0.0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to('cuda'), target.to('cuda')
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    # 调整学习率
    scheduler.step()
    
    print(f'epoch: {epoch+1}, loss: {epoch_loss / len(train_loader):.4f}')

### 5.验证模型
前面我们将原训练集分为了新的训练集和验证集，使用该验证集验证模型性能。

先处理一下验证集的数据

In [ ]:
Y_val = Y_val.values

In [ ]:
x_val_tensor = torch.FloatTensor(X_val).permute(0, 3, 1, 2).to('cuda')
y_val_tensor = torch.LongTensor(Y_val).to('cuda')

In [ ]:
x_val_tensor.shape

In [ ]:
model.eval()

with torch.no_grad():
    y_pred = model(x_val_tensor)
    y_pred = torch.argmax(y_pred, dim=1)
    accurate = \
        (y_pred == y_val_tensor).sum().item() / len(y_val_tensor)
    
    print(f'准确率为: {accurate}')

### 6. 生成提交文件
以测试集作为模型输入，并将生成的预测结果与对应图像ID按ImageID,Label的格式输出至指定文件。

In [ ]:
x_test_tensor = torch.FloatTensor(test).permute(0, 3, 1, 2).to('cuda')

In [ ]:
x_test_tensor.shape

In [ ]:
model.eval()

with torch.no_grad():
    label = model(x_test_tensor)
    label = torch.argmax(label, dim=1)
    

使用pandas库输出csv文件。

In [ ]:
df = pd.DataFrame({
        'ImageID': list(range(1, 28001)),
        'Label': label.tolist()
    })

df.to_csv('submit.csv', index=False)

### 7. 附录
比赛网址：https://www.kaggle.com/competitions/digit-recognizer
本文主要参考代码：https://www.kaggle.com/code/yassineghouzam/introduction-to-cnn-keras-0-997-top-6

源代码主要使用keras作为深度学习框架完成模型训练。